In [ ]:
from utils import *
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3
from sklearn.neural_network import MLPRegressor as MLPR
from scipy import optimize
from scipy.stats import spearmanr

In [ ]:
def runner(
    fried_num, noise=0.0, norm=False, 
    ntk_depth=[3, 100], 
    corrcoef_type='p',
    figs = False,
    diagnostic = False,
    ):

    return_vals = {
        'Dataset' : f'Friedman {fried_num}',
        'Noise': False if noise == 0.0 else True,
        'Norm' : norm,
        'Corr Type' : 'Pearson ' if corrcoef_type == 'p' else 'Spearman '
        }
    print(
        f"Friedman {fried_num}, Noise: {noise}, Norm: {norm}, corrcoef_type: {corrcoef_type}"
    )   
    
    X_init, y_init = [None, None]
    if fried_num==1:
        X_init, y_init = make_friedman1(noise=noise, random_state=18397425)
    elif fried_num==2:
        X_init, y_init = make_friedman2(noise=noise, random_state=18397425)
    elif fried_num==3:
        X_init, y_init = make_friedman3(noise=noise, random_state=18397425)
    else:
        return Exception("fried_num must be either 1, 2, or 3")
    
    datasets = processing(X_init, y_init)
    X, y, X_train, y_train = [None, datasets['orig'][1], None, datasets['orig train'][1]]

    if norm:
        X = datasets['norm'][0]
        X_train = datasets['norm train'][0]
    else:
        X = datasets['orig'][0]
        X_train = datasets['orig train'][0]


    # Kernels
    ntk_1 = (
        ConstantKernel(
            constant_value=1, 
            constant_value_bounds=(1e-9, 1e2)
        ) * 
        NTK(depth=ntk_depth[0], c=2, 
            bias=1e-1, 
            bias_bounds=(1e-9, 1e0)
        ) 
    )

    if noise != 0.0:
        ntk_1 = (
            ConstantKernel(
                constant_value=1, 
                constant_value_bounds=(1e-9, 1e2)
            ) * 
            NTK(depth=ntk_depth[0], c=2, 
                bias=1e-1, 
                bias_bounds=(1e-9, 1e0)
            ) + WhiteKernel()
        )
        



    gp_ntk = GPR(kernel=ntk_1, alpha=1e-9, normalize_y=True, n_restarts_optimizer=9, random_state=3480795)
    gp_ntk.fit(X_train, y_train)
    print('NTK 1    : ', gp_ntk.kernel_)
    mean_ntk_1 = gp_ntk.predict(X)

    ntk_2 = (
        ConstantKernel(
            constant_value=1, 
            constant_value_bounds=(1e-9, 1e2)
        ) * 
        NTK(depth=ntk_depth[1], c=2, 
            bias=1e-1, 
            bias_bounds=(1e-9, 1e0)
        ) 
    )

    if noise != 0.0:
        ntk_2 = (
            ConstantKernel(
                constant_value=1, 
                constant_value_bounds=(1e-9, 1e2)
            ) * 
            NTK(depth=ntk_depth[1], c=2, 
                bias=1e-1, 
                bias_bounds=(1e-9, 1e0)
            ) + WhiteKernel()
        )



    gp_ntk_2 = GPR(kernel=ntk_2, alpha=1e-9, normalize_y=True, n_restarts_optimizer=9, random_state=3480795)
    gp_ntk_2.fit(X_train, y_train)
    print('NTK 2    : ', gp_ntk_2.kernel_)
    mean_ntk_2 = gp_ntk_2.predict(X)




    nn = MLPR()


In [ ]:
X_init, y_init = make_friedman3(noise=noise, random_state=18397425)
datasets = processing(X_init, y_init)

X, y, X_train, y_train = [None, datasets['orig'][1], None, datasets['orig train'][1]]

if norm:
    X = datasets['norm'][0]
    X_train = datasets['norm train'][0]
else:
    X = datasets['orig'][0]
    X_train = datasets['orig train'][0]

n = X.shape[1]